In [20]:
# import necessary packages
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
import numpy as np
from collections import Counter
import seaborn as sns
import numpy as np
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, preprocessing, svm
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from collections import Counter
import random

In [21]:
#import cleaned dataset from last project
data = pd.DataFrame.from_csv('cleaned.csv')

BsmtQual = data['BsmtQual']
encodings = {'Ex': 6, 'Gd': 5, 'TA': 4, 'Fa': 3, 'Po': 2, 'NP' :1}
for key in encodings:
    BsmtQual = BsmtQual.replace(key, encodings[key])
data['BsmtQual'] = BsmtQual
    
ExterQual = data['ExterQual']
encodings = {'Ex': 6, 'Gd': 5, 'TA': 4, 'Fa': 3, 'Po': 2, 'NP' :1}
for key in encodings:
    ExterQual = ExterQual.replace(key, encodings[key])
data['ExterQual'] = ExterQual
    
ExterCond = data['ExterCond']
encodings = {'Ex': 6, 'Gd': 5, 'TA': 4, 'Fa': 3, 'Po': 2, 'NP' :1}
for key in encodings:
    ExterCond = ExterCond.replace(key, encodings[key])
data['ExterCond'] = ExterCond
    
BsmtCond = data['BsmtCond']
encodings = {'Ex': 6, 'Gd': 5, 'TA': 4, 'Fa': 3, 'Po': 2, 'NP' :1}
for key in encodings:
    BsmtCond = BsmtCond.replace(key, encodings[key])
data['BsmtCond'] = BsmtCond
    
KitchenQual = data['KitchenQual']
encodings = {'Ex': 6, 'Gd': 5, 'TA': 4, 'Fa': 3, 'Po': 2, 'NP' :1}
for key in encodings:
    KitchenQual = KitchenQual.replace(key, encodings[key])
data['KitchenQual'] = KitchenQual
    
GarageQual = data['GarageQual']
encodings = {'Ex': 6, 'Gd': 5, 'TA': 4, 'Fa': 3, 'Po': 2, 'NP' :1}
for key in encodings:
    GarageQual = GarageQual.replace(key, encodings[key])
data['GarageQual'] = GarageQual
    
ls = ['BsmtQual', 'ExterQual', 'BsmtCond', 'ExterCond', 'KitchenQual', 'GarageQual']

In [22]:
# Add target variable column 'qualAboveAverage' to dataframe 
# Specifies if house quality is above agerage (1) or below average(0)
data['qualAboveAverage'] = (data['OverallQual'] > 5).astype(int)
# Store target variable (qualAboveAverage) as y
Y = data['qualAboveAverage']
X = pd.DataFrame()
columns = data.columns 
columns_dtypes = data.dtypes
numerical_types = [np.int64, np.int32, np.float32, np.float64]
valid_col = [columns[i] for i in range(len(columns)) if columns_dtypes[i] in numerical_types]
valid_col.remove('qualAboveAverage')
valid_col.remove('OverallQual')
valid_data = pd.DataFrame(data, columns = valid_col)

enc = LabelEncoder()
for i in valid_data.columns:
    valid_data[i] = enc.fit_transform(valid_data[i])

In [23]:
"""
for column in valid_data.columns.values:
    corr = abs(valid_data[column].corr(Y))
    if corr > .4:
        print(column + ':', corr)
        X[column] = valid_data[column]
"""
X = valid_data.filter(['YearBuilt', 'YearRemodAdd', 'BsmtQual', 'TotalBsmtSF', 'GrLivArea',
       'FullBath', 'KitchenQual', 'GarageCars', 'GarageArea', 'SalePrice',
       'OverallQual', 'ExterQual', 'BsmtCond', 'BsmtUnfSF', '1stFlrSF',
       '2ndFlrSF', 'HalfBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageQual',
       'OpenPorchSF', 'MSSubClass', 'BsmtFinSF1', 'KitchenAbvGr', 'WoodDeckSF',
       'Foundation', 'HeatingQC', 'GarageYrBlt'])

YearBuilt: 0.493020979675
YearRemodAdd: 0.437861757046
ExterQual: 0.506235761975
BsmtQual: 0.521354932126
GrLivArea: 0.490761427974
FullBath: 0.487962875878
KitchenQual: 0.46337623216
GarageCars: 0.469083851474
SalePrice: 0.636310119244


In [24]:
# Calculate the baseline for Logistic Regression model
logit_baseline = max(Y.value_counts())/len(Y)
print('Logistic Regression Baseline Value:', logit_baseline)

Logistic Regression Baseline Value: 0.631434454358


In [25]:
splits = [0.1, 0.2, 0.3, 0.4, 0.5]
random_states = [98, 1560, 999, 105002, 77393]

In [26]:
# build model and fit on train set
model = LogisticRegression()
print(X.columns)

for i in range(0,5):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=splits[i], random_state=random_states[i])
    model.fit(X_train, Y_train)
    score = model.score(X_test, Y_test)
    improvement = 100*(score - logit_baseline)/logit_baseline
    print("The Logistic Regression for a train-test-split of {}-{} has an accuracy score of:\n{:.4f}"
          .format(1-splits[i], splits[i], score))
    print("Thus, this model is {:.2f}% more accurate than the baseline model."
          .format(improvement))
    print("\n")

Index(['YearBuilt', 'YearRemodAdd', 'ExterQual', 'BsmtQual', 'GrLivArea',
       'FullBath', 'KitchenQual', 'GarageCars', 'SalePrice'],
      dtype='object')
The Logistic Regression for a train-test-split of 0.9-0.1 has an accuracy score of:
0.8630
Thus, this model is 36.68% more accurate than the baseline model.


The Logistic Regression for a train-test-split of 0.8-0.2 has an accuracy score of:
0.8185
Thus, this model is 29.62% more accurate than the baseline model.


The Logistic Regression for a train-test-split of 0.7-0.3 has an accuracy score of:
0.8128
Thus, this model is 28.72% more accurate than the baseline model.


The Logistic Regression for a train-test-split of 0.6-0.4 has an accuracy score of:
0.8456
Thus, this model is 33.92% more accurate than the baseline model.


The Logistic Regression for a train-test-split of 0.5-0.5 has an accuracy score of:
0.8313
Thus, this model is 31.65% more accurate than the baseline model.


